In [0]:
# univariate LSTM example
# https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

import datetime
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model

In [0]:
def CryptoData(symbol, frequency, start=0):
    #Params: String symbol, int frequency = 300,900,1800,7200,14400,86400
    #Returns: df from first available date
    url ='https://poloniex.com/public?command=returnChartData&currencyPair='+symbol+'&end=9999999999&period='+str(frequency)+'&start='+str(start)
    df = pd.read_json(url)
    df.set_index('date',inplace=True)
    return df

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


def getPrediction(lstm, raw_seq, index, n_steps, n_features):
  x_seq = raw_seq[index-n_steps : index]
  x_seq = x_seq.reshape(1, n_steps, n_features)
  
  yhat = lstm.predict(x_seq)
  y = raw_seq[index]
  
  return x_seq, yhat, y

In [0]:
startDate = int((datetime.datetime.strptime('1/1/2018', "%d/%m/%Y").timestamp()))
testSymbol = 'USDT_BTC' #bitCoin
# frequency = 86400s which is 1 day
df = CryptoData(testSymbol, 86400, startDate)
df

In [0]:
df['close'].plot(figsize = (16,10))

In [0]:
# define input sequence and no. of features
raw_seq = df['close'].values
n_features = raw_seq.ndim
data_size = len(raw_seq)

print("n_features:", n_features) 
print("data_size:", data_size)

In [0]:
train_ratio = 0.8
train_size = round(train_ratio * data_size)
train_seq = raw_seq[:train_size]
test_seq = raw_seq[train_size:]

print("train_size:", train_size)
print("test_size:", data_size-train_size)

In [0]:
# choose a number of time steps
n_steps = 7

# split into training samples
x_train, y_train = split_sequence(train_seq, n_steps)

# reshape from [samples, timesteps] to [samples, timesteps, features]
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], n_features)

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)


In [0]:
x_train

In [0]:
y_train

In [0]:
# define the LTSM model
# The number of hidden neurons in LSTM does not directly relate to the timestep 
model = Sequential()
model.add(LSTM(32, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

plot_model(model, show_shapes=True, show_layer_names=True)

In [0]:
model.summary()
# The lstm (LSTM) Param # = g * [h(h+i) + h]
# where g is number of gates and LSTM = 4
# h = no. of LSTM hidden neurons
# i = the dimension of input (feature)
# 4 * (32 (32 + 1) + 32) = 4352

In [0]:
# fit model
model.fit(x_train, y_train, epochs=20)

In [0]:
# predict the next day
index = train_size
x_seq, yhat, y = getPrediction(model, raw_seq, index, n_steps, n_features)

print(x_seq)
print("Predicted:", yhat[0][0])
print("Actual:", y)

In [0]:
# Predict the rest of the series and comparing to the test set
# Each prediction is based on the "actual" observation from the past days
# Another way is to use multi-step ahead output or
# use the predicted values as new observations to further predict the rest of the series

predictedList = []
actualList = []

for i in range(train_size, data_size):
  x_seq, yhat, y = getPrediction(model, raw_seq, i, n_steps, n_features)
  predictedList.append(yhat[0][0])
  actualList.append(y)

for i in range(len(predictedList)):
  print("Predicted:", round(predictedList[i], 2),
       " Actual:", round(actualList[i], 2),
       " Error:", round(actualList[i]-predictedList[i], 2))

In [0]:
plt.figure(figsize=(16, 10))
plt.plot(predictedList, label="Predicted")
plt.plot(actualList, label="Actual")
plt.legend()